In [1]:
from models.DistMult import DistMult
from utils.loaders import load_data, get_onehots
import pickle
import torch
import numpy as np
from sklearn.utils import shuffle
from utils.evaluation_metrics import SRR, auprc_auroc_ap

In [2]:
with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\triplets.pickle', 'rb') as handle:
    data = pickle.load(handle)

with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\triplet_lookup.pickle', 'rb') as handle:
    lookup = pickle.load(handle)

with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\ASD_dictionary.pickle', 'rb') as handle:
    ASD_dictionary = pickle.load(handle)

with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\Sparse_dictionary.pickle', 'rb') as handle:
    BCE_dictionary = pickle.load(handle)

entities = int(len(lookup)/2)

In [3]:
number_of_batches = 5
number_of_epochs = 20
x = shuffle(data)
test_data = x[:50] #just limit data to the first n rows

In [4]:
all_objects, all_relationships, all_subjects = np.split(test_data, 3, axis=1)
all_objects = torch.LongTensor(all_objects)
all_subjects = torch.squeeze(torch.LongTensor(all_subjects))
all_relationships = torch.LongTensor(all_relationships)

In [5]:
model = DistMult(num_entities = entities, embedding_dim=100, num_relations=4)
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
for epoch in range(number_of_epochs):


        #training stage 
        model.train()
        objects, subjects, relationships  = load_data(test_data, number_of_batches)

        for index in range(number_of_batches):

            obj = torch.LongTensor(objects[index])
            rel = torch.LongTensor(relationships[index])
            subj = torch.squeeze(torch.LongTensor(subjects[index]))

            optimiser.zero_grad()
            pred = model.forward(e1 = obj, rel = rel)
            loss = model.loss(pred, subj)
            loss.backward()
            optimiser.step()


        #evaluation stage
        model.eval()
        objects, subjects, relationships  = load_data(test_data, number_of_batches)
        total_sum_reciporical_rank = torch.zeros(1)

        for index in range(number_of_batches):

            obj = torch.LongTensor(objects[index])
            rel = torch.LongTensor(relationships[index])
            targets = torch.LongTensor(subjects[index])

            predictions = model.forward(e1 = obj, rel = rel)
            srr = SRR(predictions, targets) 
            total_sum_reciporical_rank = total_sum_reciporical_rank + srr

        print('mean reciporical rank is...')
        print(total_sum_reciporical_rank / len(test_data))



mean reciporical rank is...
tensor([0.0001])
mean reciporical rank is...
tensor([0.0002])
mean reciporical rank is...
tensor([0.0005])
mean reciporical rank is...
tensor([0.0029])
mean reciporical rank is...
tensor([0.0089])
mean reciporical rank is...
tensor([0.0311])
mean reciporical rank is...
tensor([0.0556])
mean reciporical rank is...
tensor([0.0909])
mean reciporical rank is...
tensor([0.1657])
mean reciporical rank is...
tensor([0.2520])
mean reciporical rank is...
tensor([0.3543])
mean reciporical rank is...
tensor([0.4861])
mean reciporical rank is...
tensor([0.6613])
mean reciporical rank is...
tensor([0.7764])
mean reciporical rank is...
tensor([0.8411])
mean reciporical rank is...
tensor([0.8685])
mean reciporical rank is...
tensor([0.9059])
mean reciporical rank is...
tensor([0.9312])
mean reciporical rank is...
tensor([0.9725])
mean reciporical rank is...
tensor([1.])


In [11]:
one_hots = get_onehots(targets, entities)
auprc, auroc, ap = auprc_auroc_ap(one_hots, predictions)

In [12]:
print(auroc)
print(auprc)
print(ap)

0.99999744876265
0.9078535353535354
0.9107070707070707
